### Extracción de capas de interés de SIOSE
#### (Solar, Cultivos y Cultivos arbóreos)

##### Instalar paquetes no nativos

In [ ]:
!pip install geopandas pyproj

In [ ]:
!pip install fiona

##### Importar paquetes

In [32]:
import os
import glob
import geopandas as gpd
from pyproj import CRS
import zipfile
import fiona
import pandas as pd
import pickle

##### Listar todos los archivos de SIOSE, formato .gpkg para cada provincia

In [2]:
# Directorio raíz
directorio_raiz = "spatial_data/"

# Lista para almacenar los archivos .gpkg
gpkg_files = []

# Recorrer recursivamente el directorio raíz y sus subdirectorios
for directorio_actual, _, archivos in os.walk(directorio_raiz):
    # Filtrar los archivos .gpkg y agregarlos a la lista
    gpkg_files += [os.path.join(directorio_actual, archivo) for archivo in archivos if archivo.endswith(".gpkg")]

# Imprimir la lista de archivos .gpkg
print(gpkg_files)

['spatial_data/01_ALAVA.gpkg', 'spatial_data/03_ALICANTE.gpkg', 'spatial_data/04_ALMERIA.gpkg', 'spatial_data/05_AVILA.gpkg', 'spatial_data/06_BADAJOZ.gpkg', 'spatial_data/08_BARCELONA.gpkg', 'spatial_data/09_BURGOS.gpkg', 'spatial_data/10_CACERES.gpkg', 'spatial_data/12_CASTELLON.gpkg', 'spatial_data/14_CORDOBA.gpkg', 'spatial_data/15_CORUÑA.gpkg', 'spatial_data/17_GIRONA.gpkg', 'spatial_data/18_GRANADA.gpkg', 'spatial_data/20_GUIPUZCOA.gpkg', 'spatial_data/21_HUELVA.gpkg', 'spatial_data/22_HUESCA.gpkg', 'spatial_data/23_JAEN.gpkg', 'spatial_data/24_LEON.gpkg', 'spatial_data/25_LLEIDA.gpkg', 'spatial_data/26_RIOJA.gpkg', 'spatial_data/27_LUGO.gpkg', 'spatial_data/28_MADRID.gpkg', 'spatial_data/29_MALAGA.gpkg', 'spatial_data/30_MURCIA.gpkg', 'spatial_data/31_NAVARRA.gpkg', 'spatial_data/32_OURENSE.gpkg', 'spatial_data/33_ASTURIAS.gpkg', 'spatial_data/34_PALENCIA.gpkg', 'spatial_data/37_SALAMANCA.gpkg', 'spatial_data/39_CANTABRIA.gpkg', 'spatial_data/40_SEGOVIA.gpkg', 'spatial_data/41_S

##### Extracción polígonos solares

In [4]:
# Función para extraer la capa USOS y el uso 'Solar'
def extract_USOS(file_path):
    # Listar todas las capas
    layers = fiona.listlayers(file_path)
    # Seleccionar el nombre de capa que termine por '_T_USOS'
    layer_name = [layer for layer in layers if layer.endswith('_T_USOS')][0]
    # Leer la capa
    layer = gpd.read_file(file_path, layer=layer_name)
    # Extraer uso 'Solar'
    layer = layer[layer['ID_USO_MAX'].isin([2442])]
    return layer

In [5]:
# Extraer capas 
solar_polygons = [extract_USOS(file) for file in gpkg_files]

In [10]:
# Proyectar a EPSG:25830
solar_polygons = [capa.to_crs(25830) for capa in solar_polygons]

In [20]:
# Iterar sobre las capas de la lista para ver cuáles tienen más polígonos
for i, geo_database in enumerate(solar_polygons, start=1):
    num_polygons = len(geo_database)  # Obtener la cantidad de polígonos en la geodatabase actual
    print(f"La geodatabase {i} contiene {num_polygons} polígonos.")

# La que más tiene es Granada, pero nos quedamos con Badajoz que es la segunda que más tiene
# porque los polígonos de Granada creo que de media son mucho más pequeños

La geodatabase 1 contiene 1 polígonos.
La geodatabase 2 contiene 51 polígonos.
La geodatabase 3 contiene 74 polígonos.
La geodatabase 4 contiene 205 polígonos.
La geodatabase 5 contiene 491 polígonos.
La geodatabase 6 contiene 27 polígonos.
La geodatabase 7 contiene 29 polígonos.
La geodatabase 8 contiene 282 polígonos.
La geodatabase 9 contiene 48 polígonos.
La geodatabase 10 contiene 71 polígonos.
La geodatabase 11 contiene 64 polígonos.
La geodatabase 12 contiene 27 polígonos.
La geodatabase 13 contiene 502 polígonos.
La geodatabase 14 contiene 0 polígonos.
La geodatabase 15 contiene 54 polígonos.
La geodatabase 16 contiene 43 polígonos.
La geodatabase 17 contiene 40 polígonos.
La geodatabase 18 contiene 86 polígonos.
La geodatabase 19 contiene 113 polígonos.
La geodatabase 20 contiene 114 polígonos.
La geodatabase 21 contiene 2 polígonos.
La geodatabase 22 contiene 6 polígonos.
La geodatabase 23 contiene 59 polígonos.
La geodatabase 24 contiene 418 polígonos.
La geodatabase 25 cont

In [33]:
# Guardar variable de Python como archivo (pickle package)

# Ruta del archivo
ruta_archivo = "spatial_data/solar_polygons.pkl"

# Guardar la variable en el archivo
with open(ruta_archivo, "wb") as archivo:
    pickle.dump(solar_polygons, archivo)

print("Variable guardada correctamente en", ruta_archivo)

# Así podemos cargar la variable en un nuevo entorno


Variable guardada correctamente en spatial_data/solar_polygons.pkl


In [28]:
# Seleccionar Badajoz
badajoz_solar = solar_polygons[4]
badajoz_solar.head()

,PROVINCIA,MUNICIPIO,MUNICIPIO_NOMBRE,TIPO,ID_PARCELA,REFCAT,ROTULO,ID_USO_MAX,USO_DESC,HILUCS,HILUCS_DESC,SUPERF_M2,FECHA,VALORES,MEJORADO,ANO_REF,geometry
2890,6,158,ZAFRA,R,8de429ed-cf83-43e7-9795-d38939db65b6,06158A00700126,35ANATE_25PAGRI_20ESOLA_15IALIM_05RTIFC,2442,Solar,244,2_4_4_RenewableEnergyProduction,126493.59,NaN,SIGPAC,Revisión visual,2016,"POLYGON ((202668.147 4258823.065, 202662.472 4..."
21207,6,162,ZARZA (LA),R,10203d9f-229a-4438-8b34-db33e3213cd5,06162A01300026,ESOLA,2442,Solar,244,2_4_4_RenewableEnergyProduction,107.42,NaN,SIGPAC,Revisión visual,2016,"POLYGON ((215375.174 4300058.518, 215375.174 4..."
22957,6,162,ZARZA (LA),R,5951c6d3-4944-4989-8de0-f2b717c47560,06162A01300026,ESOLA,2442,Solar,244,2_4_4_RenewableEnergyProduction,69.13,NaN,SIGPAC,Revisión visual,2016,"POLYGON ((214882.400 4300242.041, 214877.297 4..."
23531,6,162,ZARZA (LA),R,734a0c97-ad1b-4b5e-8669-c670d2c80133,06162A01300026,ESOLA,2442,Solar,244,2_4_4_RenewableEnergyProduction,39.79,NaN,SIGPAC,Revisión visual,2016,"POLYGON ((215276.431 4300293.133, 215272.919 4..."
24128,6,162,ZARZA (LA),R,8bfce0e4-c674-43eb-8444-1753d4f753ed,06162A01300026,ESOLA,2442,Solar,244,2_4_4_RenewableEnergyProduction,41.77,NaN,SIGPAC,Revisión visual,2016,"POLYGON ((215071.935 4300178.404, 215071.935 4..."


In [ ]:
# Guardar
badajoz_solar.to_file("spatial_data/final_polys/badajoz_solar.shp")

##### Extracción polígonos invernaderos

In [37]:
# Continuamos sólo con Badajoz para facilitar el proceso
capas_badajoz=fiona.listlayers("spatial_data/06_BADAJOZ.gpkg")
capas_badajoz

['SAR_06_T_USOS',
 'SAR_06_T_POLIGONOS',
 'SAR_06_T_COMBINADA',
 'layer_styles',
 'LISTADO_COBERTURAS',
 'LISTADO_ATRIBUTOS',
 'LISTADO_USOS',
 'SAR_06_T_VALORES',
 'SAR_06_TABLA_PLANA']

In [36]:
# La capa que nos interesa es la COMBINADA, donde se encuentran los usos agrícolas detallados
badajoz_COMBINADA = gpd.read_file("spatial_data/06_BADAJOZ.gpkg", layer=capas_badajoz[2])

In [38]:
badajoz_COMBINADA.head()

,PROVINCIA,MUNICIPIO,MUNICIPIO_NOMBRE,ID_COMBI,ID_COBERTURA_MAX,COBERTURA_DESC,ID_USO_MAX,USO_DESC,HILUCS,HILUCS_DESC,SUPERF_M2,ANO_REF,geometry
0,6,158,ZAFRA,5de44888-14f9-4ebd-ab3e-bff31475c391,290,Prados,6310,Áreas naturales terrestres,631,6_3_1_LandAreasNotInOtherEconomicUse,573.90,2016.0,"POLYGON ((197706.978 4258445.625, 197706.091 4..."
1,6,158,ZAFRA,7c2d83d1-7e4c-40d6-b30f-df7d23fb6b73,290,Prados,6310,Áreas naturales terrestres,631,6_3_1_LandAreasNotInOtherEconomicUse,417.40,2016.0,"POLYGON ((197742.555 4258364.863, 197743.345 4..."
2,6,158,ZAFRA,7df9be49-7d1d-4c9d-8170-eb1bb3115e77,290,Prados,6310,Áreas naturales terrestres,631,6_3_1_LandAreasNotInOtherEconomicUse,19559.78,2016.0,"POLYGON ((197921.441 4260514.572, 197923.402 4..."
3,6,158,ZAFRA,36cfc045-f62b-4601-bdb0-c70a9c5e5a94,290,Prados,6310,Áreas naturales terrestres,631,6_3_1_LandAreasNotInOtherEconomicUse,3447.08,2016.0,"POLYGON ((194952.762 4260855.728, 194970.235 4..."
4,6,158,ZAFRA,ea250535-7ccb-4a95-842c-0eb8d9900627,290,Prados,6310,Áreas naturales terrestres,631,6_3_1_LandAreasNotInOtherEconomicUse,23848.64,2016.0,"POLYGON ((195767.129 4261160.032, 195777.910 4..."


In [39]:
# Extraer capas 
badajoz_greenhouse = badajoz_COMBINADA[badajoz_COMBINADA['ID_USO_MAX'].isin([1110])]

In [40]:
# Proyectar a EPSG:25830
badajoz_greenhouse = badajoz_greenhouse.to_crs(25830)

##### Extracción polígonos cultivos arbóreos

In [ ]:
badajoz_treecrops = badajoz_COMBINADA[badajoz_COMBINADA['ID_USO_MAX'].isin([1110])]to_crs(25830)